#### Problem Statement : BoomBikes aspires to understand the demand for shared bikes among the people after this ongoing quarantine situation ends across the nation due to Covid-19. They have planned this to prepare themselves to cater to the people's needs once the situation gets better all around and stand out from other service providers and make huge profits.
#### You are required to model the demand for shared bikes with the available independent variables. It will be used by the management to understand how exactly the demands vary with different features. They can accordingly manipulate the business strategy to meet the demand levels and meet the customer's expectations
#### In this notebook, we will build Linear regression model to predict "Rental Bike Demands" Using an appropriate Varibles. 

#### Contributor : Hariharan J

In [3]:
# Import libraries

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 

import statsmodels
import statsmodels.api as sm

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

# i) Data Visualization

- Perform EDA to understand various variables.
- Check the correlation between the variables.

In [4]:
# Read the day csv file 
bikeDemands = pd.read_csv("day.csv")
bikeDemands.head()

instant      dteday  season  yr  mnth  holiday  weekday  workingday  \
0        1  01-01-2018       1   0     1        0        6           0   
1        2  02-01-2018       1   0     1        0        0           0   
2        3  03-01-2018       1   0     1        0        1           1   
3        4  04-01-2018       1   0     1        0        2           1   
4        5  05-01-2018       1   0     1        0        3           1   

   weathersit       temp     atemp      hum  windspeed  casual  registered  \
0           2  14.110847  18.18125  80.5833  10.749882     331         654   
1           2  14.902598  17.68695  69.6087  16.652113     131         670   
2           1   8.050924   9.47025  43.7273  16.636703     120        1229   
3           1   8.200000  10.60610  59.0435  10.739832     108        1454   
4           1   9.305237  11.46350  43.6957  12.522300      82        1518   

    cnt  
0   985  
1   801  
2  1349  
3  1562  
4  1600

In [ ]:
# Find the number of row & Column 
bikeDemands.shape

In [ ]:
# Find the non null and Datatypes of variables 
bikeDemands.info()

In [ ]:
# percentage of missing values in each column
round(100*(bikeDemands.isnull().sum()/len(bikeDemands.index)), 2).sort_values(ascending=False)

In [ ]:
# row-wise null count percentage
round((bikeDemands.isnull().sum(axis=1)/122)*100,2).sort_values(ascending=False)

Insights: - There are NO MISSING/NULL values either in columns or rows.

In [ ]:
bikeDuplicate = bikeDemands.copy()

- No change in shape so using the bikeDuplicate dataframe for further processing and original copy is maintained. 

In [ ]:
bikeDuplicate['dteday'] =  pd.to_datetime(bikeDuplicate['dteday'],format='%d-%m-%Y')
bikeDuplicate['dteday'].dtype

In [ ]:
# We derive two new column derivedyear and month from dteday
bikeDuplicate['derivedyear'] = pd.DatetimeIndex(bikeDuplicate['dteday']).year
bikeDuplicate['month'] = pd.DatetimeIndex(bikeDuplicate['dteday']).month
bikeDuplicate.head()

bikeDuplicate.describe()

In [ ]:
# Dropping the dteday,instant,casual and registered columns.

# Based on data dictionary, the following variables can be ignored.
# - instant : Its only an index value 
# - dteday : This has the date, Since we already have seperate derived columns for 'year' & 'month' we can ignore this column too. 
# - casual & registered : Bike booked by different categories of customers. Since our objective is to find the total count of bikes and not by specific category, we will ignore these two columns. 

bikeDuplicate.drop(['dteday','instant','casual','registered','derivedyear','mnth'],axis=1,inplace=True)
bikeDuplicate.head()

In [ ]:
# Here we renamed few improper column names to proper names for better understanding.
# Our Target variable is entireCount
bikeDuplicate.rename(columns={'hum':'humidity','cnt':'entireCount','atemp':'feeltemp','yr':'year'},inplace=True)
bikeDuplicate.head()

## Visualise the numerical Data

In [ ]:
# Visualising Numeric Variables using pair plot 
sns.pairplot(data=bikeDuplicate, x_vars=['temp','feeltemp','humidity','windspeed','entireCount'],y_vars=['temp','feeltemp','humidity','windspeed','entireCount'])

There is Linear relationship between temp,feeltemp and entirecount using pairplot by above.

In [ ]:
# Visualising Numeric Variables with target variable using pair plot 
sns.pairplot(data=bikeDuplicate, x_vars=['temp','feeltemp','humidity','windspeed'], y_vars='entireCount')

In [ ]:
corr_df= bikeDuplicate[['temp','humidity','windspeed','entireCount']].corr()
f, ax = plt.subplots(figsize=(14, 9))
sns.heatmap(corr_df, 
            xticklabels=corr_df.columns.values,
            yticklabels=corr_df.columns.values,annot= True)
plt.show()

We can see that there are some independent variables look positively correlated to the 'count' variable.

Bike rentals are more correlated to temperature

## Visualise the Categorical Variables

In [ ]:
##Relationship between categorical and continuous variable

plt.figure(figsize=(20, 12))

plt.subplot(2,4,1)
sns.boxplot(x = 'year', y = 'entireCount', data = bikeDuplicate)

plt.subplot(2,4,2)
sns.boxplot(x = 'holiday', y = 'entireCount', data = bikeDuplicate)

plt.subplot(2,4,3)
sns.boxplot(x = 'workingday', y = 'entireCount', data = bikeDuplicate)

plt.subplot(2,4,4)
sns.boxplot(x = 'month', y = 'entireCount', data = bikeDuplicate)

plt.subplot(2,4,5)
sns.boxplot(x = 'weathersit', y = 'entireCount', data = bikeDuplicate)

plt.subplot(2,4,6)
sns.boxplot(x = 'season', y = 'entireCount', data = bikeDuplicate)

plt.subplot(2,4,7)
sns.boxplot(x = 'weekday', y = 'entireCount', data = bikeDuplicate)

plt.show()

Insights:  7 categorical variables vs one target Variable (entireCount) using box plot

The inference derived is 

- holiday: Almost 97.6% of the bike booking were happening when it is not a holiday which means this data is clearly biased. This indicates, holiday CANNOT be a good predictor for the dependent variable.

- workingday: Almost 69% of the bike booking were happening in ‘workingday’ with a median of close to 5000 booking (for the period of 2 years). This indicates, workingday can be a good predictor for the dependent variable. ---

- mnth: Almost 10% of the bike booking were happening in the months 5,6,7,8 & 9 with a median of over 4000 booking per month. This indicates, mnth has some trend for bookings and can be a good predictor for the dependent variable.

- weathersit: Almost 67% of the bike booking were happening during ‘weathersit1 with a median of close to 5000 booking (for the period of 2 years). This was followed by weathersit2 with 30% of total booking. This indicates, weathersit does show some trend towards the bike bookings can be a good predictor for the dependent variable.

- season: Almost 32% of the bike booking were happening in season3 with a median of over 5000 booking (for the period of 2 years). This was followed by season2 & season4 with 27% & 25% of total booking. This indicates, season can be a good predictor for the dependent variable.

- weekday: weekday variable shows very close trend (between 13.5%-14.8% of total booking on all days of the week) having their independent medians between 4000 to 5000 bookings. This variable can have some or no influence towards the predictor. I will let the model decide if this needs to be added or not. 

## Univariate Analysis

In [ ]:
# just create a copy of dataframe for string mapping for season,month,weathersit and weekday 
bikeRepresentation = bikeDuplicate.copy()

In [ ]:
#Mapping variables season, month, weathersit, weekday
bikeRepresentation['season']=bikeRepresentation.season.map({1: 'spring', 2: 'summer',3:'fall', 4:'winter' })
bikeRepresentation['month']=bikeRepresentation.month.map({1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May',6:'June',7:'July',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec'})
bikeRepresentation['weathersit']=bikeRepresentation.weathersit.map({1: 'Clear',2:'Mist + Cloudy',3:'Light Snow',4:'Snow + Fog'})
bikeRepresentation['weekday']=bikeRepresentation.weekday.map({0:'Sun',1:'Mon',2:'Tue',3:'Wed',4:'Thu',5:'Fri',6:'Sat'})
bikeRepresentation['workingday']=bikeRepresentation.workingday.map({1: 'Working', 0: 'Holiday'})

bikeRepresentation.head()

### 1. Season

In [ ]:
#Relation between season and count of bike rentals
sns.barplot('season','entireCount',data=bikeRepresentation,palette="rocket",)
plt.show()

- Bike Rentals are more during fall season than summer 

### 2. Weathersit

1: Clear, Few clouds, Partly cloudy, Partly cloudy
2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog


In [ ]:
#Relation between weather and count of bike rentals
sns.barplot('weathersit','entireCount',palette="muted",data=bikeRepresentation)
plt.show()         

- Bike rentals are more during partly cloudy weathersit.

### 3. Year

In [ ]:
#Relation between Year and count of bike rentals
sns.barplot('year','entireCount',palette="muted",data=bikeRepresentation)
plt.show() 

- Bike rentals are more in 2019 than 2018 (year wise)

### 4. Month based on year

In [ ]:
plt.figure(figsize=(10,5))
sns.barplot('month','entireCount',hue='year',data=bikeRepresentation,palette='Paired')
plt.show()

- Bike rentals are more in 2019 than 2018 (Month wise)

### 5. Working Day 

In [ ]:
#Relation between workingDay and count of bike rentals
sns.barplot('workingday','entireCount',palette="muted",data=bikeRepresentation)
plt.show()

- Bike Rentals are more in working day.

### 6. WeekDay:

In [ ]:
#Relation between WeekDay and count of bike rentals
bikeRepresentation.groupby('weekday')['entireCount'].max().plot(kind='bar')

- Bike rentals are more in saturday and sunday

###  Numerical variables - temp, feeltemp,humidity and windspeed - so we can draw scatter plotter 

### 7. Temperature 

In [ ]:
sns.regplot(x='temp',y='entireCount', data = bikeRepresentation)

- Bike Rentals are observed at higher temperatures.

### 8. Feel Temperature 

In [ ]:
sns.regplot(x='feeltemp',y='entireCount', data = bikeRepresentation)

- Bike Rentals are observed at higher 'Feel-Like' temperatures.

### 9. Humidity

In [ ]:
sns.regplot(x='humidity',y='entireCount', data = bikeRepresentation)

- Bike rentals more at high humidity

### 10. WindSpeed Temperature 

In [ ]:
sns.regplot(x='windspeed',y='entireCount', data = bikeRepresentation)

- Wind speeds increase with a greater temperature difference. Wind speed near the surface is most highly correlated with the temperature

In [ ]:
sns.distplot(bikeRepresentation['entireCount'])

- The entire count of casual and registered user are normall distributed. 

# ii) Data Preparation

• Create dummy variables for all the categorical features.
• Divide the data to train & Test.
• Perform Scaling.
• Divide data into dependent & Independent variables.

## 1. Creating Dummy Variables

In [ ]:
bikeRepresentation['season']=bikeRepresentation['season'].astype('category')
bikeRepresentation['weathersit']=bikeRepresentation['weathersit'].astype('category')
bikeRepresentation['month']=bikeRepresentation['month'].astype('category')
bikeRepresentation['weekday']=bikeRepresentation['weekday'].astype('category')
bikeRepresentation.info()


In [ ]:
bikeRepresentation.head()

In [ ]:
# This code does 3 things:
# 1) Create Dummy variable
# 2) Drop original variable for which the dummy was created
# 3) Drop first dummy variable for each set of dummies created.
bikeRepresentation = pd.get_dummies(bikeRepresentation, drop_first=True)

# We will visualise the changes after the dummy creation.
bikeRepresentation.info()

In [ ]:
bikeRepresentation.shape

In [ ]:
bikeRepresentation.head()

## 2. Splitting the Data into Training and Testing Sets

In [ ]:
bike_train, bike_test = train_test_split(bikeRepresentation,train_size=0.7, random_state= 100)

In [ ]:
#Rows and columns after split
print(bike_train.shape)
print(bike_test.shape)

## 3. Rescaling the Features

It is important to have all the variables on the same scale for the model to be easily interpretable. We can use standardization or normalization so that the units of the coefficients obtained are all on the same scale.

There are two common ways of rescaling:

Min-Max scaling (Normalisation):Between 0 and 1

Standardisation :mean-0, sigma-1

In [ ]:
scaler = MinMaxScaler()

#Create a list of numeric variables
num_vars=['temp','feeltemp','humidity','windspeed','entireCount']

In [ ]:
bike_train[num_vars].head()

In [ ]:
#Fit on data
bike_train[num_vars] = scaler.fit_transform(bike_train[num_vars])

In [ ]:
bike_train.head()

In [ ]:
bike_train.describe()

In [ ]:
# Let's check the correlation coefficients to see which variables are highly correlated in train data set 
plt.figure(figsize = (25,20))
sns.heatmap(bike_train.corr(), annot = True, cmap="YlGnBu")
plt.show()

## 4. Dividing into X and Y sets for the model building

In [ ]:
y_train = bike_train.pop('entireCount')
X_train = bike_train

# iii) Data Modeling & Evaluation

• Create Linear Regression model using mixed approach (RFE & VIF/p-value).
• Check the various assumptions.
• Check the Adjusted R-Square for both train & Test data.
• Report the final model.

 ## Build Linear Model

- We will be using the LinearRegression function from SciKit Learn for its compatibility with Recursive Feature Elimination (RFE)

### Step 1. Recursive Feature Elimination (RFE)

In [ ]:
# Importing RFE and LinearRegression

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

In [ ]:
# Running RFE with the output number of the variable equal to 15
lm = LinearRegression()
lm.fit(X_train, y_train)

rfe = RFE(lm, 15)            # running RFE . Also passing 15 for 15 variable selection
rfe = rfe.fit(X_train, y_train)

In [ ]:
list(zip(X_train.columns,rfe.support_,rfe.ranking_))

In [ ]:
# These are top 15 variables selected 
columns = X_train.columns[rfe.support_]
columns

In [ ]:
# These are remaining variables not considered.
X_train.columns[~rfe.support_]

In [ ]:
# Creating X_test dataframe with RFE selected variables
X_train_rfe = X_train[columns]

### Step 2. Build Model using 'STATS MODEL'

### we should decided which variable should be dropped based on below two criteria .

1. Significance of p value (Remove variable whose p valuse > 0.05)
2. VIF (usually VIF < 5 are considered)

### Checking the Variance Inflation Factor (VIF)

Sometimes pairwise correlations aren't enough
Instead of just one variable, the independent variable might depend upon a combination of other variables
VIF calculates how well one independent variable is explained by all the other independent variables combined

The common heuristic we follow for the VIF values is:

 Greater than 10:  Definitely high VIF value and the variable should be eliminated.
 
 Greater than 5:  Can be okay, but it is worth inspecting.
 
 Less than 5: Good VIF value. No need to eliminate this variable.

### we can follow this rule 

High P value and High VIF - Directly eliminate the variable 

High - Low:
   1. High P and Low VIF (Remove these first) 
   2. Low P  and High VIF (Remove these after, so that VIF may decrease)
   
Low p value and Low VIF  - Consider the variable  

### Model 1 (Initial Model)

In [ ]:
# Adding a constant variable 
import statsmodels.api as sm  
X_train_rfe = sm.add_constant(X_train_rfe)

# Running the linear model 
lm = sm.OLS(y_train,X_train_rfe).fit()

print(lm.summary())

#Drop the constant term B0
X_train_rfe = X_train_rfe.drop(['const'], axis=1)

 # Calculate the VIFs for the new model
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
X = X_train_rfe
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif


### month_Jan is insignificant in presence of other variables due to high p-value and low VIF; can be dropped

In [ ]:
#Drop month_Jan
X_train_new1 = X_train_rfe.drop(["month_Jan"], axis = 1)

### Model 2  without 'month_Jan'

In [ ]:
#Build a model
X_train_lm1 = sm.add_constant(X_train_new1)
lm1 = sm.OLS(y_train,X_train_lm1).fit()
print(lm1.summary())

#Drop the constant term B0
X_train_lm1 = X_train_lm1.drop(['const'], axis=1)
# Calculate the VIFs for the new model

vif = pd.DataFrame()
X = X_train_new1
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

### 'humidity' variable can be dropped as its insignificant by looking at very high VIF

In [ ]:
#Drop humidity
X_train_new2 = X_train_lm1.drop(["humidity"], axis = 1)

### Model 3 without 'humidity'

In [ ]:
#Build a model
X_train_lm2 = sm.add_constant(X_train_new2)
lm2 = sm.OLS(y_train,X_train_lm2).fit()
print(lm2.summary())

#Drop the constant
X_train_lm2=X_train_lm2.drop(['const'],axis=1)

# Calculate the VIFs for the new model
vif = pd.DataFrame()
X = X_train_new2
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

### Holiday variable seems to be insignificant, by looking at p value and low VIF. We can drop it

In [ ]:
#Drop the holiday column
X_train_new3=X_train_lm2.drop(['holiday'],axis=1)

### Model 4 without holiday

In [ ]:
# Adding a constant variable 
X_train_lm3 = sm.add_constant(X_train_new3)
lm3 = sm.OLS(y_train,X_train_lm3).fit()
print(lm3.summary())

#Drop constant 
X_train_lm3=X_train_lm3.drop(['const'],axis=1)

# Calculate the VIFs for the new model
vif = pd.DataFrame()
X = X_train_new3
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

### Windspeed seems to be insignificant,by looking at high VIF and negative correlation with count. Lets drop it

### Model 5 without windspeed

In [ ]:
#Drop July
X_train_new4= X_train_lm3.drop(['windspeed'], axis=1)

In [ ]:
#Build a model
X_train_lm4=sm.add_constant(X_train_new4)
lm4=sm.OLS(y_train,X_train_lm4).fit()
print(lm4.summary())

#Drop constant
X_train_lm4= X_train_lm4.drop(['const'], axis=1)

# Calculate the VIFs for the new model
vif = pd.DataFrame()
X =X_train_new4
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

### Month_July column can be dropped due to its p value and low VIF

In [ ]:
#Drop July
X_train_new5=X_train_lm4.drop(['month_July'], axis=1)

In [ ]:
#Building a model
X_train_lm5= sm.add_constant(X_train_new5)
lm5=sm.OLS(y_train,X_train_lm5).fit()
print(lm5.summary())

#Drop the constant
X_train_lm7=X_train_lm5.drop(['const'],axis=1)
# Calculate the VIFs for the new model


vif = pd.DataFrame()
X = X_train_new5
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

### The model is lm5 which is obtained by removing month_Jan, humidity, holiday, windspeed  and month_July variables from the RFE support columns

Finally we have our model ! The VIFs and p-values both are within an acceptable range. So we go ahead and make our predictions using this model only.

- The p-value for each term tests the null hypothesis that the coefficient is equal to zero (no effect). A low p-value (< 0.05) indicates that you can reject the null hypothesis.

- A rule of thumb commonly used in practice is if a VIF is > 10, you have high multicollinearity. In our case, with values less than 5, we are in good shape, and can proceed with our regression

- R-squared measures the strength of the relationship between your model and the dependent variable on a convenient 0 – 100% scale. And we have the R-square value of 0.826 or 82.6%

- The adjusted R-squared adjusts for the number of terms in the model. And we got it around 0.82 or 82%

# VALIDATE ASSUMPTIONS

### Step 3. Residual Analysis

#### i) Error terms are normally distributed with mean zero (not X, Y)

In [ ]:
#X_train_lm5=sm.add_constant(X_train_lm5)
#X_train_lm5.columns
X_train_lm5

In [ ]:
#y train predicted
y_train_pred = lm5.predict(X_train_lm5)

In [ ]:
residual = y_train - y_train_pred

fig = plt.figure()
plt.figure(figsize=(14,7))
sns.distplot((residual))

plt.title('Distribution of Error Terms using histogram', fontsize = 20) 
plt.xlabel('Errors', fontsize = 18) 
plt.show()

#### Insights:
- From the above histogram, we could see that the Residuals are normally distributed. Hence our assumption for Linear Regression is valid.

#### ii) There is a linear relationship between X and Y

In [ ]:
bike_new = bikeRepresentation[[ 'temp', 'feeltemp', 'humidity', 'windspeed','entireCount']]
sns.pairplot(bike_new, diag_kind='kde')
plt.show()

#### Insights:

- Using the pair plot, we could see there is a linear relation between temp and feeltemp variable with the predictor ‘entireCount’.

#### iii) There is No Multicollinearity between the predictor variables¶

In [ ]:
vif = pd.DataFrame()
X = X_train_new5
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

#### Insights: 
-  VIF calculation we could find that there is no multicollinearity existing between the predictor variables, as all the values are within permissible range of below 5

### Step 4. Predication on Test Set.

In [ ]:
#Create a list of numeric variables
num_vars=['temp','feeltemp','humidity','windspeed','entireCount']

#Fit on data
bike_test[num_vars] = scaler.transform(bike_test[num_vars])
bike_test.head()

In [ ]:
#Dividing into X_test and y_test
y_test = bike_test.pop('entireCount')
X_test = bike_test
X_test.describe()

In [ ]:
#Columns
X_train_new5.columns

In [ ]:
# Now let's use our model to make predictions.

# Creating X_test_new dataframe by dropping variables from X_test
X_test_new = X_test[X_train_new5.columns]

# Adding a constant variable 
X_test_new1 = sm.add_constant(X_test_new)
X_test_new1.head()

In [ ]:
# Making predictions
y_pred = lm5.predict(X_test_new1)

### Find R-squared for Test set

In [ ]:
#Evaluate R-square for test
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)

### Find Adjusted R-Squared for Test set

In [ ]:
# Evaluate Adjusted R^2 for test 

#adj r2=1-(1-R2)*(n-1)/(n-p-1)
#n =sample size , p = number of independent variables

Adj_r2=1-(1-0.8115083)*(11-1)/(11-1-1)
print(Adj_r2)

### Step 5 Evaluation on Test Set

In [ ]:
# Plotting y_test and y_pred to understand the spread using Scatter plot

fig = plt.figure()
plt.figure(figsize=(15,8))
plt.scatter(y_test,y_pred,color='brown')
plt.title('y_test vs y_pred', fontsize = 20)         # Plot heading 
plt.xlabel('y_test', fontsize=18)                          # X-label
plt.ylabel('y_pred', fontsize=16)                          # Y-label

In [ ]:
#Regression plot

plt.figure(figsize=(15,8))
sns.regplot(x=y_test, y=y_pred, ci=68, fit_reg=True,scatter_kws={"color": "brown"}, line_kws={"color": "blue"})

plt.title('y_test vs y_pred', fontsize=20)              # Plot heading 
plt.xlabel('y_test', fontsize=18)                          # X-label
plt.ylabel('y_pred', fontsize=16)                          # Y-label
plt.show()

# Final Discussion

### We can see that the equation of our best fitted line is:
$ entireCount =  0.4914 * temp +   0.0916 * month Sep + 0.0645 * weekday Sat + 0.0527 * season summer + 0.0970 * season winter + 0.2334 * Year + 0.0566 * workingday Working  - 0.03041 * weathersit Light Snow	 - 0.0786 * weathersit Mist + Cloudy - 0.065 * season spring $

###  Comparison between Train and Test model after analysis:
 
 Train - R^2 : 0.826
 
 Test - R^2: 0.8115
 
 Difference in R^2 between train and test: 1.5%

 Train - Adjusted R^2 : 0.82
 
 Test - Adjusted R^2: 0.790564
 
 Difference in adjusted R^2 between Train and test: 3.15%
 
 Since adusted R^2 is less than which is less than 5% difference in R^2, we can conclude that it is the best model

### Assumptions of Linear Regression:

The error terms are normally distributed.
The training and testing accuracy are nearly equal hence there is no Overfit/Underfit situation.
The predicted values have linear relationship with the actual values.

### Analysis on the problem:

- Temperature variable is having the highest coefficient 0.4914, which means if the temperature increases by one unit the number of bike rentals increases by 0.4914 units.

- Some variables with negative coefficients. A negative coefficient suggests that as the independent variable increases, the dependent variable tends to decrease. we see below varibales with negative coefficient.

    1. - 0.03041 * weathersit Light Snow	
    2. - 0.0786  * mist weathersit Mist + Cloudy 
    3. - 0.065   * season spring  
    
- The coefficient value signifies how much of the dependent variable changes given a one-unit shift in the independent variable while holding other variables in the model constant.

### Business Goals:

- Temperature play a vital role for making decision for the Organisation. 
- Season and Weathersit too impacts the business, So consider that too for better business outcomes
- Working days as they have good influence on bike rentals. So providing offers for working professional can improve the business 
- We can see demand for bikes was more in 2019 than 2018


# Feature Engineering Purpose 

In [ ]:
# Creating a new dataframe for just providing visibility for future vision
bike_fep=bikeDemands.copy()

In [ ]:
bike_fep['dteday'] =  pd.to_datetime(bike_fep['dteday'],format='%d-%m-%Y')
bike_fep['dteday'].dtype

In [ ]:
#Week number:
#We will extract the week number (1-52) from the date (dteday) column provided
bike_fep['weekno']=bike_fep['dteday'].dt.week


In [ ]:
#Quarter:
#We will calculate the Quarter from the month (mnth) column provided
bike_fep['qtr']= bike_fep['mnth']//4+1


In [ ]:
#Registered to Casual user ratio:
#We will calculate the registered to casual user ratio from the columns (registered & casual) provided
bike_fep['rc_ratio']=round((bike_fep['registered']/bike_fep['casual']),1)


In [ ]:
#Feeling to Actual temperature ratio:
#We will calculate the Feeling (atemp) to actual (temp) temperature ratio from the columns provided
bike_fep['temp_ratio']=round(bike_fep['atemp']/bike_fep['temp'],1)


In [ ]:
#Weather Comfort Index:
#An arbitrary index of the suitability of environmental conditions to physical activity.

# Comfort Index = (temperature + relative humidity)/4.

bike_fep['wci']=(bike_fep['atemp']+bike_fep['hum'])/4
bike_fep.head()

## The above data frame shows new values which can be used for future engineering, now we are not considering this data frame. So stay chill. 